In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import UnstructuredExcelLoader

# 엑셀 파일 로드 및 문서로 변환
xlsx_path = 'test용사전엑셀.xlsx'
xlsx_loader = UnstructuredExcelLoader(xlsx_path, mode="elements")
docs = xlsx_loader.load()

# Embeddings 및 문서 분할 설정
embeddings = OpenAIEmbeddings()
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in docs])

# FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(documents=semantic_chunks, embedding=embeddings)

# 번역을 위한 PromptTemplate 설정
translation_prompt = PromptTemplate(
    input_variables=["retrieved_text", "query"],
    template="""
다음은 한국어 텍스트와 관련된 참고 자료입니다:
참고 자료: {retrieved_text}

위 자료를 기반으로, 아래 한국어 문장을 영어로 번역해주세요.
문장: {query}
번역:
"""
)

trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

translation_chain = LLMChain(
    llm=trans_llm, 
    prompt=translation_prompt
    )

retriever = vectorstore.as_retriever()


C:\Users\201\AppData\Local\Temp\ipykernel_12996\851973997.py:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\201\AppData\Local\Temp\ipykernel_12996\851973997.py:35: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
C:\Users\201\AppData\Local\Temp\ipykernel_12996\851973997.py:37: LangC

번역 결과: Zinrabit likes bananas and Zingorani.


In [3]:
query = "산독끼는 바나나와 꼬라니를 좋아해."
retrieved_docs = retriever.get_relevant_documents(query)
print("지금 뭐가 있는데?", retrieved_docs )

# 검색 결과를 결합하여 참고 자료로 활용
retrieved_text = " ".join([doc.page_content for doc in retrieved_docs])

result = translation_chain.run({"retrieved_text": retrieved_text, "query": query})
print("번역 결과:", result)

지금 뭐가 있는데? [Document(id='c64a12d9-8f7b-41fc-a4d6-46e5c8bf68dc', metadata={}, page_content='한국어 영어 개 dog 고양이 cat 바나나 banana 커피 coffee 안경 glasses 바위 rock 시험 test 저장 save 긍정적인 positve 산독끼 Zinrabit 꼬라니 Zingorani')]
번역 결과: Zinrabit likes bananas and Zingorani.
